# Creating ``Chalice``
***Follow this [link](https://aws.github.io/chalice/quickstart.html) for an easy instulation guide***

* Open a terminal, or perfered user-interface
* Verify that chalice is installed

In [11]:
!chalice --version

chalice 1.21.4, python 3.8.3, windows 10


* Use the command **``mkdir .aws``** to make a directory to store AWS keys

In [12]:
!mkdir .aws

A subdirectory or file .aws already exists.


* Use the **``touch .aws/config``** to create a document inside of the **``.aws``** file

In [13]:
!touch .aws/config

* Launch the **``config``** file with your perfered user-inter face
* Store your keys in the file like so
    * **``aws_access_key_id="PUBLIC_KEY"``**
    * **``aws_secret_access_key="SECRET_KEY"``**
    * **``region=SET_REGION``**

* Create a new directory 
    * ``mkdir "Name of directory"``<br>
(optional, used for organization)

In [14]:
!mkdir running_chalice

A subdirectory or file running_chalice already exists.


* ``cd`` into the new directory

In [15]:
!cd running_chalice

* Use the chalice command to create a new project
    * **``chalice new-project "NAME_OF_PROJECT"``**

In [16]:
!chalice new-project Algo_trade

Directory already exists: Algo_trade
Aborted!


* ``cd`` into the new project
* Locate the **``app.py``** and launch it with perfered user-interface    

---
# Connecting Alpaca

***Using the ``app.py`` file launched above we are going to connect to Alpaca***<br>
***Start by importing the modules and libraries need***

In [17]:
from chalice import Chalice
import requests, json

* Create the headers Alpaca/Chalice needs 

In [18]:
app = Chalice(app_name='YOUR_PROJECT_NAME')
API_KEY="YOUR_PUBLIC_ALPACA_KEY"
SECRET_KEY="YOUR_SECRET_ALPACA_KEY"
BASE_URL = "https://paper-api.alpaca.markets"
ORDERS_URL = "{}/v2/orders".format(BASE_URL)
HEADERS = {'APCA-API-KEY-ID': API_KEY, 'APCA-API-SECRET-KEY': SECRET_KEY}

* Creating the braket order to sit on the resting URL

In [19]:
@app.route('/buy_stock', methods=['POST'])
# ^^^Naming the URL^^^

def buy_stock(): 
# ^^^Creating the function

    request = app.current_request 
# ^^^Receving the information about the request^^^
    
    webhook_message = request.json_body 
# ^^^The json request being sent in (in our case from TRADING_VIEW)^^^
    
    data = { 
        "symbol": webhook_message["ticker"], 
#    ^^^ticker symbol that alert sends us^^^
        "qty": 1, 
#   ^^^number of shares^^^
        "side": "buy", 
#    ^^^buying the stock^^^
        "type": "limit", 
#  ^^^setting limit order^^^
        "limit_price":webhook_message["close"], 
#       ^^^setting buying price^^^
        "time_in_force": "gtc", 
#        ^^^Length of order^^^
        "order_class": "bracket", 
#         ^^^type of order^^^
        "take_profit": {
            "limit_price": webhook_message["close"] * 1.05}, 
#             ^^^Setting profit taking^^^
        "stop_loss": {
            "stop_price": webhook_message["close"] * 0.98,} 
#             ^^^Setting stop loss^^^
    }
 
    r = requests.post(ORDERS_URL, json=data, headers=HEADERS) 
#         ^^^Sending Request^^^

    response = json.loads(r.content) 
#    ^^^Recieving Request Response^^^
    print(response)
    print(response.content)

    return {
        'message':'I bought the stock', 
#   ^^^Returning personal message (Optional)^^^

        'webhook': webhook_message 
#   ^^^Returning the json message (Optional)^^^
    }

### <U>**Your final ``app.py`` file should look like this**</U><br>
![app.py](Images/app.png)

* Save the **``app.py``** file 
* In the terminal deploy the chalie
    * ``chalice deploy``
* Coppy the "Rest" URL provided<br>
![URL](Images/chalice_deploy.png)

# ***Creating Alerts and adding the weebhook with ``trading_view``***